# Evaluating a Strategy using Bootstrap Metrics

Bootstrap metrics can help us to more thoroughly evaluate a trading strategy, as we will see below.

[In the last notebook](https://pybroker.com/en/latest/notebooks/2.%20Backtesting%20a%20Strategy.html), we wrote a trading strategy that we backtested. Here is the implementation again:

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

def buy_low(ctx):
    if ctx.long_pos():
        return
    if len(ctx.low) >= 2 and ctx.close[-1] < ctx.low[-2]:
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        ctx.hold_bars = 3
        
def short_high(ctx):
    if ctx.short_pos():
        return
    if len(ctx.high) >= 2 and ctx.close[-1] > ctx.high[-2]:
        ctx.sell_shares = 100
        ctx.hold_bars = 2

And as before, we configure a new [Strategy](https://pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) instance:

In [2]:
config = StrategyConfig(initial_cash=500_000, bootstrap_sample_size=100)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)

This time, the ```Strategy``` is configured with a [bootstrap_sample_size](https://pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.bootstrap_sample_size) of ```100``` (the default is ```1_000```). Next, the ```Strategy``` is backtested again, but now with bootstrap metrics enabled by default:

In [3]:
strategy.add_execution(buy_low, ['AAPL', 'MSFT'])
strategy.add_execution(short_high, ['TSLA'])
result = strategy.backtest(train_size=0.5)
result.metrics_df

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2019-08-29 04:00:00 to 2022-02-25 05:00:00


100% (629 of 629) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:03 

Finished backtest: 0:00:06


,name,value
0,trade_count,402.000000
1,initial_value,500000.000000
2,end_value,598779.070000
3,total_profit,245219.260000
4,total_loss,-146440.190000
5,max_drawdown,-54071.400000
6,max_drawdown_pct,-8.346543
7,win_rate,0.547264
8,loss_rate,0.452736
9,avg_profit,2229.266000


When looking at ```initial_value``` and ```end_value``` above, it appears that we have successfully implemented a profitable trading strategy on our first attempt! But how can we be so sure that those results are repeatable and were not just a fluke? We can gain more confidence in our results by computing metrics using the boostrap method.

The basic idea behind the bootstrap method is to repeatedly compute a metric on random samples drawn from the backtest results. Then the metric is computed on each random sample and the average is taken. By computing the metric on thousands of random samples, a more robust and accurate estimate of the metric is obtained.

## Confidence Intervals

**PyBroker** uses the bootstrap method to compute confidence intervals for the [Profit Factor](https://pybroker.com/en/latest/reference/pybroker.eval.html#pybroker.eval.EvalMetrics.profit_factor) and [Sharpe Ratio](https://en.wikipedia.org/wiki/Sharpe_ratio):

In [4]:
result.bootstrap.conf_intervals

lower     upper
name          conf                     
Profit Factor 97.5%  0.172890  1.718227
              95%    0.288190  1.718227
              90%    0.422445  1.671160
Sharpe Ratio  97.5%  0.117041  0.381007
              95%    0.149927  0.381007
              90%    0.188996  0.381007

Specifically, **PyBroker** used the [bias corrected and accelerated (BCa) bootstrap method](https://blogs.sas.com/content/iml/2017/07/12/bootstrap-bca-interval.html) to compute the confidence intervals above. The returns used for the bootstrap were sampled per-bar rather than per-trade to maximize the information captured by those metrics.

We can see that the lower bounds of both the Profit Factor and Sharpe Ratio are below 1. This is not a good sign, and shows that our strategy is not reliably profitable!

## Maximum Drawdown

Still, we continue by looking at bootstrap metrics for maximum drawdown: 

In [5]:
result.bootstrap.drawdown_conf

,amount,percent
conf,,
99.9%,-407160.97,-49.077537
99%,-319118.01,-40.886733
95%,-253816.91,-34.147239
90%,-216466.82,-29.909448


Shown above are the probabilities that the drawdown will not exceed the computed values, which are given in cash amounts and percentages of portfolio equity. Like the Profit Factor and Sharpe Ratio, these confidence levels were computed using per-bar returns obtained from the backtest's out-of-sample results.

The bootstrapped max drawdown of ```49%``` at a ```99.9%``` confidence level is much worse than the ```8.35%``` we saw in our original results!

Hopefully, this example gives you a sense of the importance of using randomized tests to analyze (and scrutinize) the performance of your trading strategy.

[The next notebook will go over how to use ranking and position sizing in your trading strategies](https://pybroker.com/en/latest/notebooks/4.%20Ranking%20and%20Position%20Sizing.html).